# Kafka Consumer

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2,org.postgresql:postgresql:42.1.1 pyspark-shell'

### Import dependencies


In [2]:
#Stream processing
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json

In [3]:
#Text processing
import preprocessor as p
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

### Text cleaning

In [4]:
stop_words = set(stopwords.words("english")) #create a set of stopwords
stop_words = stop_words.union(set(("im", "thats","rt"))) #add these stopwords
stop_words = stop_words - set("not") #remove not from the stopwords

p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI, p.OPT.SMILEY) #params to remove from the text

def preprocess(text):
    
    #remove url, mentions, smiley and emojis
    text = p.clean(text)
    
    #to lowercase
    text = text.lower()
    
    #remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    #removing stopwords and stem each word
    tokens = word_tokenize(text)
    
    #is_noun = lambda pos: pos[:2] == 'NN'
    text = [i for i in tokens if not i in stop_words]
    #text = [word for (word, pos) in nltk.pos_tag(tokens) if is_noun(pos)] 
    
    return ' '.join(text)

### Create Spark context and SQL context

In [5]:
sc = SparkContext(appName="StreamProcessorPyspark").getOrCreate()
sqlContext = SQLContext(sc)
sc.setLogLevel("WARN")

### Create Streaming Context

In [6]:
ssc = StreamingContext(sc, 10)
ssc.checkpoint('ssc_checkpoint')

### Connect to Kafka


In [7]:
ip = 'localhost'
kafkaParams = {"metadata.broker.list": ip+':9092', "auto.offset.reset": 'largest'}
myStream = KafkaUtils.createDirectStream(ssc, ['DIC'], kafkaParams)

In [8]:
import psycopg2

host = "ec2-54-217-206-65.eu-west-1.compute.amazonaws.com"
database = "d9bf9qompakvh6"
user = "cuapumoqmbkafk"
password = "a510ef81c98a872ea8e47b58e7e044fb5f204f56b7019a581aa3b4f9223498f8"

#conn = psycopg2.connect(host= host,database= database, user= user, password= password, port = 5432)
#cur = conn.cursor()

### Process DStream

In [9]:
tweets = myStream.map(lambda item: json.loads(item[1]))
tweets = tweets.map(lambda x: ((x['tag'], x['date']), preprocess(x['tweet']).split()))

pairs = tweets.flatMapValues(lambda x: x)
words = pairs.map(lambda x: ((x[0][0], x[0][1], x[1]), 1))                     

In [10]:
def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
        
    return sum(newValues, runningCount)

counts = words.updateStateByKey(updateFunction)

### Store results into DB

In [11]:
def sendPartition(iter):
    for record in iter:
        conn = psycopg2.connect(host= host,database= database, user= user, password= password, port = 5432)
        cur = conn.cursor()

        treshold = 2
        print(record[1])
        if (record[1] > treshold):
            sql = "INSERT INTO word (tag, date, word, frequency) VALUES (%s, %s, %s, %s) "
            cur.execute(sql, (record[0][0],record[0][1],record[0][2],record[1], record[0][0],record[0][1],record[0][2],record[1]))
            conn.commit()

counts.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))

In [12]:
ssc.start()
ssc.awaitTermination(timeout=60)

KeyboardInterrupt: 